In [2]:
import torch
from d2l import torch as d2l

devices=d2l.try_all_gpus()
def run(x):
    return [x.mm(x) for _ in range(50)]

x_gpu1=torch.rand(size=(4000,4000),device=devices[0])
x_gpu2=torch.rand(size=(4000,4000),device=devices[1])

In [3]:
run(x_gpu1)
run(x_gpu2) # 预热设备
torch.cuda.synchronize(devices[0])
torch.cuda.synchronize(devices[1])

with d2l.Benchmark('GPU1 time'):
    run(x_gpu1)
    torch.cuda.synchronize(devices[0])

with d2l.Benchmark('GPU2 time'):
    run(x_gpu2)
    torch.cuda.synchronize(devices[1])

GPU1 time: 1.4481 sec
GPU2 time: 1.4627 sec


In [4]:
with d2l.Benchmark('GPU1 and GPU2 time'):
    run(x_gpu1)
    run(x_gpu2)
    torch.cuda.synchronize()

GPU1 and GPU2 time: 1.4544 sec


In [5]:
def copy_to_cpu(x,non_blocking=False):
    return [y.to('cpu',non_blocking=non_blocking) for y in x]

with d2l.Benchmark('在GPU1上运行'):
    y=run(x_gpu1)
    torch.cuda.synchronize()

with d2l.Benchmark('复制到CPU'):
    y_cpu=copy_to_cpu(y)
    torch.cuda.synchronize()

在GPU1上运行: 1.4940 sec
复制到CPU: 2.5485 sec


In [6]:
with d2l.Benchmark('在GPU1上运行并复制到CPU'):
    y=run(x_gpu1)
    y_cpu=copy_to_cpu(y,non_blocking=True)
    torch.cuda.synchronize()

在GPU1上运行并复制到CPU: 1.7964 sec
